In [1]:
import pickle
import chromadb
import numpy as np
from tqdm.auto import tqdm
from pprint import pprint

c:\Users\mmart\Programming\VectorDB_playground\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file = open("../data/filtered_data.pickle", "rb")
data = pickle.load(file)
file.close()

# chroma_client = chromadb.Client()

chroma_client = chromadb.PersistentClient(path="../data/chroma_dbs/")
chroma_client.list_collections()

[Collection(name=title_vector), Collection(name=abstract_vector)]

In [3]:
data.fillna("", inplace=True)
data.head(3)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cat_freq,journal_freq,date_freq
19,704.002,Patrick Roudeau,"The BABAR Collaboration, B. Aubert, et al",Measurement of the Hadronic Form Factor in D0 ...,"21 pages, 13 postscript figures, submitted to ...","Phys.Rev.D76:052005,2007",10.1103/PhysRevD.76.052005,"BABAR-PUB-07/015, SLAC-PUB-12417",hep-ex,,The shape of the hadronic form factor f+(q2)...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2015-06-30,"[[The BABAR Collaboration, , ], [Aubert, B., ]]",17311,2.0,536
2032,704.2033,Anwar Shiekh Dr.,A.Y. Shiekh,The Quantum Interference Computer: an experime...,"6 pages, 1 figure","Int. Jour. of Theo. Phys., 47, 2176, 2008",10.1007/s10773-008-9664-7,,quant-ph,,An experiment is proposed to test the interf...,"[{'version': 'v1', 'created': 'Mon, 16 Apr 200...",2009-11-13,"[[Shiekh, A. Y., ]]",66088,2.0,25981
2087,704.2088,Shimul Akhanjee,Shimul Akhanjee,Exact longitudinal plasmon dispersion relation...,"4 pages, 1 figure. Important typos and errors ...","Phys. Rev. B 76, 165129 (2007)",10.1103/PhysRevB.76.165129,,cond-mat.str-el,,We derive the exact longitudinal plasmon dis...,"[{'version': 'v1', 'created': 'Tue, 17 Apr 200...",2011-11-09,"[[Akhanjee, Shimul, ]]",25554,2.0,992


In [4]:
data = data[
    [
        "id",
        "title",
        "abstract",
        "authors",
        "journal-ref",
        "categories",
        "comments",
        "update_date",
    ]
]

data.drop_duplicates(subset="id", inplace=True)
data.drop_duplicates(subset="title", inplace=True)

documents = list(data["title"].values)
ids = list(data["id"].astype("str").values)
metedata = list(data.to_dict(orient="records"))

In [5]:
collection = chroma_client.create_collection(name="arxiv_collection")
# chroma_client.get_collection(name="arxiv_collection")

In [6]:
batch_size = 500
for i in tqdm(range(0, len(ids), batch_size)):
    collection.add(
        # embeddings=[[1.2, 2.3, 4.5], [6.7, 8.2, 9.2]], # could add embed if they are already computed!
        documents=documents[i : i + batch_size],
        metadatas=metedata[i : i + batch_size],
        ids=ids[i : i + batch_size],
    )

 27%|██▋       | 7/26 [02:46<07:33, 23.85s/it]


KeyboardInterrupt: 

In [15]:
results = collection.query(
    query_texts=[
        "Topic Space Trajectories: A case study on machine learning literature"
    ],
    n_results=3,
)

In [16]:
pprint(results)

{'data': None,
 'distances': [[0.0, 0.9720746874809265, 0.9970589280128479]],
 'documents': [['Topic Space Trajectories: A case study on machine learning '
                'literature',
                'Leveraging Large Language Models for Topic Classification in '
                'the Domain\n'
                '  of Public Affairs',
                'BATS: A Spectral Biclustering Approach to Single Document '
                'Topic Modeling\n'
                '  and Segmentation']],
 'embeddings': None,
 'ids': [['2010.12294', '2306.02864', '2008.02218']],
 'metadatas': [[{'abstract': '  The annual number of publications at '
                             'scientific venues, for example,\n'
                             'conferences and journals, is growing quickly. '
                             'Hence, even for researchers it\n'
                             'becomes harder and harder to keep track of '
                             'research topics and their progress.\n'
               